In [13]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.chrome.options import Options

from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,TimeoutException
from bs4 import BeautifulSoup

import time

import pandas as pd


In [14]:

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=options)
#driver.get("https://www.google.com")
#sy4vM
#el = driver.find_element(By.CLASS_NAME, "sy4vM")
#el.click()

In [15]:
def search_for_place_reviews(name,driver):

    driver.get("https://www.google.com")
    #sy4vM
    try:
        el = driver.find_element(By.CLASS_NAME, "sy4vM")
        el.click()
    except NoSuchElementException:
        print("No cookie to be clicked")
        
    search_box = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//input[@class='gLFyf gsfi']")))

    #search_box = driver.find_element(By.XPATH, "//input[@class='gLFyf gsfi']")
    search_box.clear()
    search_box.send_keys(name)
    try:
        search_box.send_keys(Keys.ENTER)
    except StaleElementReferenceException:
        print("QUA")

    try:
        WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH,"//a[@data-async-trigger='reviewDialog']"))).click()        
    except NoSuchElementException:
        print("No Element Found")
        return False
    except TimeoutException:
        print("No link to review found")
        return False

    return True

In [16]:
def get_reviews_html(driver):

    #RMCqNd
    wait = WebDriverWait(driver,timeout=10)
    wait.until(EC.element_to_be_clickable((By.CLASS_NAME,"review-dialog-list")))
    #reviewSort
    scroll_list = driver.find_element(By.CLASS_NAME,"review-dialog-list")
    
    for i in range(12):
        driver.execute_script(
                    'arguments[0].scrollTop = arguments[0].scrollHeight', 
                        scroll_list
                    )

    driver.execute_script(
                    'arguments[0].scrollTop = 0', 
                        scroll_list
                    )
    #EXPAND ALTRO
    #review_boxes = WebDriverWait(scroll_list, 20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME,"gws-localreviews__google-review")))
    #review_boxes = scroll_list.find_elements(By.CLASS_NAME,"gws-localreviews__google-review")
    
    
    try:
        altri = WebDriverWait(scroll_list, 2).until(EC.presence_of_all_elements_located((By.CLASS_NAME,"review-more-link")))
        for altro in altri:
            altro.click()
        #altro = review.find_element(By.CLASS_NAME,"review-more-link").click()
        #driver.execute_script("arguments[0].click();", altro)
    except TimeoutException:
        print("no altri")
    parent = scroll_list.find_element(By.XPATH,"parent::*")
    child = parent.find_elements(By.XPATH,".//*")[0]
    return child.get_attribute("innerHTML"),scroll_list.get_attribute("innerHTML")


In [17]:
#RMCqNd
#contiene 1 div vuoto e 1 div con classe:

##gws-localreviews__general-reviews-block
# questa div contiene 10 recensioni
def get_score(html_to_parse):
    
    soup = BeautifulSoup(html_to_parse,"html.parser")

    reviews_score_container = soup.find("div",{"class":"review-score-container"})
    mean_score = reviews_score_container.find("span",attrs={"class":"Aq14fc"}).text
    return mean_score

def get_reviews(html_to_parse):
    soup = BeautifulSoup(html_to_parse,"html.parser")

    id_sort = soup.find("div",attrs={'id':"reviewSort"})

    div_packs = id_sort.find_all("div",attrs={'class':"RMCqNd"})
    review_boxes = []
    for div in div_packs:
        review_boxes.extend(div.find_all("div",attrs={"class":"gws-localreviews__google-review"}))
    reviews = []
    for review_div in review_boxes:
        
        reviews.append(review_div)
    return reviews


In [18]:
schools = pd.read_csv("../vivoscuola/data/school_list_cleaned.csv",header=None)

schools = schools.values.tolist()

data = []

scores = []

for school in schools:
    
    id = school[-2]
    

    name = school[-3]
    miur_code = school[-4]
    if(search_for_place_reviews(name + " " + school[1] ,driver)):
    
        html_score,html_reviews = get_reviews_html(driver)
        score = get_score(html_score)
        reviews = get_reviews(html_reviews)
        scores.append([id,miur_code,score,name])
        for review in reviews:
            div1 = review.find("div",attrs={"class":"jxjCjc"})
            div2 = div1.find("div",attrs={"style":"vertical-align:top"})
            div3 = div2.find("div",attrs={"class":"Jtu6Td"})
            span = div3.find("span")

            if(span.text != ""):
                data.append([id,miur_code,span.text,name])
                print(span.text)
    print(len(scores))

    

No link to review found
0
No cookie to be clicked
QUA
no altri
1
No cookie to be clicked
QUA
no altri
2
No cookie to be clicked
QUA
No link to review found
2
No cookie to be clicked
QUA
No link to review found
2
No cookie to be clicked
QUA
No link to review found
2
No cookie to be clicked
QUA
No link to review found
2
No cookie to be clicked
QUA
No link to review found
2
No cookie to be clicked
QUA
no altri
3
No cookie to be clicked
QUA
No link to review found
3
No cookie to be clicked
QUA
No link to review found
3
No cookie to be clicked
QUA
no altri
Liceo con una significativa tradizione educativa  in Val di Fiemme intitolato al gruppo di ragazzi tedeschi dissidenti dal nazismo di Hitler e giustiziati giovanissimi
4
No cookie to be clicked
QUA
no altri
Liceo con una significativa tradizione educativa  in Val di Fiemme intitolato al gruppo di ragazzi tedeschi dissidenti dal nazismo di Hitler e giustiziati giovanissimi
5
No cookie to be clicked
QUA
no altri
Bellissimo ben organizzato c

In [19]:
import csv
with open("data/institutes_scores.csv","w",newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["school_id","miur_code","score","school_name"])
    writer.writerows(scores)

with open("data/institutes_reviews.csv","w",newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["school_id","miur_code","text_review","school_name"])
    writer.writerows(data)